In [1]:
# 📦 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# 📁 데이터 불러오기
df = pd.read_parquet('prepro_data.parquet')
df1 = df[df['호선'] == 2].copy()
cols = ['연도', '호선','상하구분','AWS지점코드']
for col in cols:
    df1[col] = df1[col].astype('category')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15187582 entries, 0 to 15653609
Data columns (total 25 columns):
 #   Column    Dtype         
---  ------    -----         
 0   시간        datetime64[ns]
 1   호선        int64         
 2   역번호       int32         
 3   역명        category      
 4   상하구분      category      
 5   AWS지점코드   int64         
 6   기온        float32       
 7   풍향        float32       
 8   풍속        float32       
 9   일강수량      float32       
 10  시간강수량     float32       
 11  상대습도      float32       
 12  체감온도      float32       
 13  혼잡도       float32       
 14  승차총승객수    float32       
 15  하차총승객수    float32       
 16  미세먼지      float32       
 17  연도        int32         
 18  월         int32         
 19  일         int32         
 20  시         int32         
 21  요일        int32         
 22  주말        int32         
 23  공휴일       int32         
 24  일사량_측정여부  int32         
dtypes: category(2), datetime64[ns](1), float32(11), int32(9), int64(2)
memory u

In [3]:
# 시 (0~23 기준)
df1['시_sin'] = np.sin(2 * np.pi * df1['시'] / 24)
df1['시_cos'] = np.cos(2 * np.pi * df1['시'] / 24)

# 요일 (0~6 기준, 월~일)
df1['요일_sin'] = np.sin(2 * np.pi * df1['요일'] / 7)
df1['요일_cos'] = np.cos(2 * np.pi * df1['요일'] / 7)

# 월, 일 (1~12, 1~31 기준)
le = LabelEncoder()
df['월'] = le.fit_transform(df['월'])
df['일'] = le.fit_transform(df['일'])

# 1. float64 → float32
float_cols = df1.select_dtypes(include=['float64']).columns
df1[float_cols] = df1[float_cols].astype('float32')

# 2. int64 → int32
int_cols = df1.select_dtypes(include=['int64']).columns
df1[int_cols] = df1[int_cols].astype('int32')

# 3. object(문자열) → category
obj_cols = df1.select_dtypes(include=['object']).columns
df1[obj_cols] = df1[obj_cols].astype('category')    
    
df1 = df1.drop(['시', '요일'], axis=1)
df1.info()

# ✅ [1] 피처 구성 및 전처리 --------------------------------------------------------

# 🎯 타겟 변수 '혼잡도' 제거하여 X 생성
X = df1.drop(['혼잡도', '시간'], axis=1)

# 🔤 범주형 변수 원-핫 인코딩 (첫 번째 카테고리는 drop)
o_h_e_x = pd.get_dummies(X, columns=['연도', '역명', '호선', '상하구분', 'AWS지점코드'], drop_first=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1868722 entries, 950460 to 12006245
Data columns (total 27 columns):
 #   Column    Dtype         
---  ------    -----         
 0   시간        datetime64[ns]
 1   호선        category      
 2   역번호       int32         
 3   역명        category      
 4   상하구분      category      
 5   AWS지점코드   category      
 6   기온        float32       
 7   풍향        float32       
 8   풍속        float32       
 9   일강수량      float32       
 10  시간강수량     float32       
 11  상대습도      float32       
 12  체감온도      float32       
 13  혼잡도       float32       
 14  승차총승객수    float32       
 15  하차총승객수    float32       
 16  미세먼지      float32       
 17  연도        category      
 18  월         int32         
 19  일         int32         
 20  주말        int32         
 21  공휴일       int32         
 22  일사량_측정여부  int32         
 23  시_sin     float32       
 24  시_cos     float32       
 25  요일_sin    float32       
 26  요일_cos    float32       
dtypes: category

In [4]:
# 🔢 X 정규화 (0~1 범위로 스케일링)
mm = MinMaxScaler()
mm_x = mm.fit_transform(o_h_e_x)

# 🎯 y 정규화 (표준화: 평균 0, 표준편차 1)
y = df1[['혼잡도']]  # 2D로 유지
ss = StandardScaler()
ss_y = ss.fit_transform(y)

In [5]:
# ✅ [2] 시계열 학습/검증 데이터 분할 --------------------------------------------------

# 전체의 80%를 학습용, 나머지 20%를 검증용으로 사용
split_index = int(len(mm_x) * 0.8)

train_x = mm_x[:split_index]
train_y = ss_y[:split_index]

val_x = mm_x[split_index:]
val_y = ss_y[split_index:]

# (3) Attention Layer 추가 (Self-Attention 또는 Bahdanau)

In [ ]:
# ✅ [3] 시계열 데이터 생성 -----------------------------------------------------------
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# ✅ ModelCheckpoint 콜백 추가 ---------------------------------------
checkpoint = ModelCheckpoint(
    filepath='best_model_Attention_layer.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

# 과거 21시간 데이터를 기반으로 예측
sequence_length = 21
batch_size = 32

# TimeseriesGenerator를 통해 시계열 학습/검증셋 생성
train_gen = TimeseriesGenerator(train_x, train_y, length=sequence_length, batch_size=batch_size)
val_gen   = TimeseriesGenerator(val_x, val_y, length=sequence_length, batch_size=batch_size)

# ✅ [4] Attention 기반 LSTM 모델 정의 --------------------------------------------------------

# 커스텀 Attention Layer 정의
class Attention(tf.keras.layers.Layer):
    def __init__(self):
        super(Attention, self).__init__()

    def call(self, inputs):
        # inputs: (batch_size, time_steps, hidden_size)
        score = tf.nn.softmax(tf.keras.backend.sum(inputs, axis=2), axis=1)
        context = tf.keras.backend.batch_dot(score, inputs, axes=1)
        return context

# 모델 구조 정의 (Functional API 사용)
input_layer = Input(shape=(sequence_length, train_x.shape[1]))
lstm = LSTM(64, return_sequences=True)(input_layer)
attn = Attention()(lstm)
flatten = Flatten()(attn)
output = Dense(1)(flatten)

model3 = Model(inputs=input_layer, outputs=output)

# 모델 컴파일
model3.compile(loss='mse', optimizer=Adam(0.001))

# 콜백 설정
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5)

# 모델 학습
history3 = model3.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=[
        # early_stop, 
        reduce_lr,
        checkpoint
    ]
)

In [ ]:
# 📊 학습 및 검증 손실 시각화
plt.rcParams['font.family']='Malgun Gothic'
plt.figure(figsize=(10, 5))
plt.plot(history3.history['loss'], label='Train Loss')
plt.plot(history3.history['val_loss'], label='Validation Loss')
plt.title('Attention Layer 추가 Model Loss During Training')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
plt.show()

# (4) Layer Normalization 또는 BatchNormalization

In [ ]:
# ✅ [3] 시계열 데이터 생성 -----------------------------------------------------------
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# ✅ ModelCheckpoint 콜백 추가 ---------------------------------------
checkpoint = ModelCheckpoint(
    filepath='best_model_normalizaion.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

# 과거 21시간 데이터를 기반으로 예측
sequence_length = 21
batch_size = 32

# TimeseriesGenerator를 통해 시계열 학습/검증셋 생성
train_gen = TimeseriesGenerator(train_x, train_y, length=sequence_length, batch_size=batch_size)
val_gen   = TimeseriesGenerator(val_x, val_y, length=sequence_length, batch_size=batch_size)

# ✅ [4] Attention 기반 LSTM 모델 정의 --------------------------------------------------------
from tensorflow.keras.layers import LayerNormalization

model4 = Sequential([
    Input(shape=(sequence_length, train_x.shape[1])),
    LSTM(64, return_sequences=True),
    LayerNormalization(),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1)
])


# 모델 컴파일
model4.compile(loss='mse', optimizer=Adam(0.001))

# 콜백 설정
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5)

# 모델 학습
history4 = model4.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=[
        # early_stop, 
        reduce_lr,
        checkpoint
    ]
)

In [ ]:
# 📊 학습 및 검증 손실 시각화
plt.rcParams['font.family']='Malgun Gothic'
plt.figure(figsize=(10, 5))
plt.plot(history4.history['loss'], label='Train Loss')
plt.plot(history4.history['val_loss'], label='Validation Loss')
plt.title('Layer Normalization 추가 Model Loss During Training')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
plt.show()

# 모델 비교

In [ ]:
# ✅ [5] 예측 및 평가 ---------------------------------------------------------------
from tensorflow.keras.models import load_model
loaded_model_3 = load_model('best_model_Attention_layer.h5')
loaded_model_4 = load_model('best_model_normalizaion.h5')

# 검증셋에 대해 예측 수행
pred_scaled_3 = loaded_model_3.predict(val_gen)
pred_scaled_4 = loaded_model_4.predict(val_gen)

# 정규화 해제 (역정규화) → 실제 혼잡도 단위로 복원
pred_3 = ss.inverse_transform(pred_scaled_3)
pred_4 = ss.inverse_transform(pred_scaled_4)

# 실제 y도 시계열 슬라이싱 기준에 맞게 자르기
true = ss.inverse_transform(val_y[sequence_length:])

In [21]:
# 평가 지표 계산                            # R² Score
rmse_3 = mean_squared_error(true, pred_3, squared=False)  # RMSE
r2_3 = r2_score(true, pred_3)                             # R² Score
rmse_4 = mean_squared_error(true, pred_4, squared=False)  # RMSE
r2_4 = r2_score(true, pred_4)                             # R² Score

# 결과 출력
print('✅Attention Layer 추가')
print(f" 검증 RMSE: {rmse_3:.4f}")
print(f" R² Score: {r2_3:.4f}")
print()
print('✅Layer Normalization')
print(f" 검증 RMSE: {rmse_4:.4f}")
print(f" R² Score: {r2_4:.4f}")

✅batch size = 32
 검증 RMSE: 4.0428
 R² Score: 0.9709

✅batch size = 64
 검증 RMSE: 4.6141
 R² Score: 0.9620

✅batch size = 128
 검증 RMSE: 4.8596
 R² Score: 0.9579

✅batch size = 256
 검증 RMSE: 4.6932
 R² Score: 0.9607


Epoch 23: val_loss improved from 0.03989 to 0.03952, saving model to best_model_batchsize_32.h5  
Epoch 15: val_loss improved from 0.05258 to 0.05148, saving model to best_model_batchsize_64.h5  
Epoch 19: val_loss improved from 0.05784 to 0.05711, saving model to best_model_batchsize_128.h5  
Epoch 29: val_loss improved from 0.05363 to 0.05326, saving model to best_model_batchsize_256.h5